# Step 4 : Model Evaluation 

MLFlow integration step 

In [1]:
!pip install python-dotenv

In [12]:
import os 
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')
load_dotenv()

True

### Change Working Directory 

In [3]:
import os 
print("Current Directory : ")
%pwd

Current Directory : 


'd:\\Data Science\\Codes\\02 MLOPs\\00 MLOPs Project\\research'

In [4]:
os.chdir("../")
print("Current Directory : ")
%pwd

Current Directory : 


'd:\\Data Science\\Codes\\02 MLOPs\\00 MLOPs Project'

In [5]:
import tensorflow as tf 
model = tf.keras.models.load_model("artifacts/training/model.keras") 

# Data Class (Structure)
Blue print of data variables needed for this step of pipeline 

In [6]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model : Path
    training_data : Path 
    all_params : dict 
    mlflow_uri : str 
    params_image_size : list
    params_batch_size : int 

# Configuration Manager 

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories,save_json 

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_PATH_YAML,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self)-> EvaluationConfig:

        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/asadullahcreative/Kidney-Disease-Classification-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config

# Evaluation Class

In [9]:
import tensorflow as tf
from pathlib import Path 
import mlflow
import mlflow.keras
from urllib.parse import urlparse 

d:\Users\Asadullah Core\Apps\AS\envs\tf_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config  # Store config with model path, image size, batch size

    def _valid_generator(self):
        # Image preprocessing settings
        datagenerator_kwargs = dict(
            rescale=1./255,  # Normalize pixel values from [0,255] to [0,1]
            validation_split=0.30  # Reserve 30% of data for validation
        )

        # Data loading settings
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],  # Resize images to (height, width)
            batch_size=self.config.params_batch_size,  # Number of images per batch
            interpolation="bilinear"  # Image resizing method
        )

        # Create ImageDataGenerator for validation data
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        # Load validation images from directory
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,  # Path to data folder
            subset="validation",  # Use validation split
            shuffle=False,  # Keep order for consistent evaluation
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        """Load saved Keras model from disk"""
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        # Load the trained model
        self.model = self.load_model(self.config.path_of_model)
        # Prepare validation data
        self._valid_generator()
        # Evaluate model and get loss/metrics
        self.score = self.model.evaluate(self.valid_generator)  # Fixed: was 'model', should be 'self.model'

        # self.save_score()

    def save_score(self):
        scores = {"loss":self.score[0],"accuracy":self.score[1]}
        save_json(path=Path("scores.json"),data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)

        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme 

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss":self.score[0],"accuracy":self.score[1]}
            )

            # Model registry does not work with file store 
            if tracking_uri_type_store != "file":

                # Register the model 
                mlflow.keras.log_model(self.model,"model")
            else:
                mlflow.keras.log_model(self.model,"model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e 

[2026-02-10 14:38:04,005: INFO: common: YAML file loaded successfully: config\config.yaml:]
[2026-02-10 14:38:04,013: INFO: common: YAML file loaded successfully: params.yaml:]
[2026-02-10 14:38:04,021: INFO: common: Directory created at: artifacts:]
Found 139 images belonging to 2 classes.
9/9 ━━━━━━━━━━━━━━━━━━━━ 72s 8s/step - accuracy: 0.5036 - loss: 0.6775


2026/02/10 14:39:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/10 14:39:25 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


[2026-02-10 17:34:29,881: WARNING: connectionpool: Retrying (Retry(total=6, connect=7, read=7, redirect=7, status=7)) after connection broken by 'SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2427)')': /asadullahcreative/Kidney-Disease-Classification-Project.mlflow/api/2.0/mlflow-artifacts/artifacts/94fbb4106caa4cf9b83acddd46a86b37/models/m-538cd24e096143cd8cafdb03dfb96535/artifacts/data/model.keras:]
[2026-02-10 17:34:34,117: WARNING: connectionpool: Retrying (Retry(total=5, connect=6, read=7, redirect=7, status=7)) after connection broken by 'NameResolutionError("HTTPSConnection(host='dagshub.com', port=443): Failed to resolve 'dagshub.com' ([Errno 11001] getaddrinfo failed)")': /asadullahcreative/Kidney-Disease-Classification-Project.mlflow/api/2.0/mlflow-artifacts/artifacts/94fbb4106caa4cf9b83acddd46a86b37/models/m-538cd24e096143cd8cafdb03dfb96535/artifacts/data/model.keras:]
[2026-02-10 18:46:12,941: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read

MlflowException: API request to https://dagshub.com/asadullahcreative/Kidney-Disease-Classification-Project.mlflow/api/2.0/mlflow-artifacts/artifacts/94fbb4106caa4cf9b83acddd46a86b37/models/m-538cd24e096143cd8cafdb03dfb96535/artifacts/data/model.keras failed with exception HTTPSConnectionPool(host='dagshub.com', port=443): Max retries exceeded with url: /asadullahcreative/Kidney-Disease-Classification-Project.mlflow/api/2.0/mlflow-artifacts/artifacts/94fbb4106caa4cf9b83acddd46a86b37/models/m-538cd24e096143cd8cafdb03dfb96535/artifacts/data/model.keras (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2427)')))